In [14]:
# 흰색의 면적을 계산해주는 프로그램
# 이분법 사진 폴더와, 가로변의 길이를 미리 구해다 주세요.

In [15]:
"""
폴더 안의 흑백(또는 회색조) 이미지를 읽어
검은색(0)으로 표시된 영역의 면적을 mm^2 단위로 계산합니다.

- 한글 경로/파일명 완전 지원 (imdecode/tofile 사용)
- 이미지가 완전한 흑백이 아니어도 임계값으로 이분화(기본 128)
- '검은색'을 관심영역(전경)으로 가정
- 옵션: 가장 큰 연결 성분만 측정(LARGEST_ONLY)
- 결과: CSV 저장 + (옵션) 마스크 미리보기 PNG 저장

사용예)
python measure_black_area.py

코드 상단의 사용자 설정 섹션만 수정하세요.
"""

import os
import sys
import csv
from pathlib import Path
import numpy as np
import cv2

# =========================
# 사용자 설정
# =========================
# 1) 이미지 폴더 경로 (예: r"C:\데이터\imgs")
FOLDER_PATH = r"C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\250901_영상\피규어용\Aloe\aloe_ver2_300-700\Binary"

# 2) 사진의 '가로 길이' (실세계, mm)
REAL_WIDTH_MM = 66  # 예: 50 mm

# 3) 이분화 임계값 (0~255 사이, 이미지가 완전 바이너리가 아니면 사용)
THRESH = 128

# 4) 가장 큰 덩어리(연결성분)만 측정할지 여부

# 5) 마스크 미리보기 이미지를 저장할지 여부
SAVE_MASK_PREVIEW = True

In [16]:


# 6) 처리 대상 확장자
EXTS = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff"}

# =========================
# 유틸: 한글 경로 대응 imread/imwrite
# =========================
def imread_unicode(path, flags=cv2.IMREAD_UNCHANGED):
    """한글/공백 경로 대응 이미지 읽기"""
    data = np.fromfile(str(path), dtype=np.uint8)
    img = cv2.imdecode(data, flags)
    return img

def imwrite_unicode(path, img, params=None):
    """한글/공백 경로 대응 이미지 쓰기"""
    ext = Path(path).suffix
    if not ext:
        ext = ".png"
        path = str(path) + ext
    result, enc = cv2.imencode(ext, img, params or [])
    if not result:
        raise IOError("이미지 인코딩 실패")
    enc.tofile(str(path))

# =========================
# 메인 로직
# =========================
def binarize_to_black_foreground(gray, thresh=128):
    """
    입력: 회색조(gray).
    출력: 이진 이미지(binary).
    - 픽셀값 > thresh -> 255(흰색 배경)
    - 픽셀값 <= thresh -> 0(검은색 전경: 우리가 측정할 영역)
    """
    if len(gray.shape) == 3:
        gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, thresh, 255, cv2.THRESH_BINARY)
    return binary  # 0=검은색(전경), 255=흰색(배경)

def largest_component_mask(binary):
    """
    바이너리에서 가장 큰 연결 성분을 0(검정=관심) / 255(배경) 규약으로 반환
    (우리 규약: 검정이 전경이므로, 성분 내부를 검정으로 만들기)
    """
    # 성분 추출을 위해 전경을 흰색으로 바꿔 임시 처리
    # 현재 binary: 전경=0, 배경=255
    fg = (binary == 0).astype(np.uint8)  # 전경=1, 배경=0
    num, labels, stats, _ = cv2.connectedComponentsWithStats(fg, connectivity=8)
    if num <= 1:
        return binary.copy()

    # 0은 배경 라벨, 1~num-1이 성분
    areas = stats[1:, cv2.CC_STAT_AREA]
    max_idx = 1 + np.argmax(areas)
    largest = (labels == max_idx).astype(np.uint8)  # 1: largest, 0: others

    # 다시 규약에 맞게: 전경=0(검정), 배경=255(흰)
    out = np.where(largest == 1, 0, 255).astype(np.uint8)
    return out

def process_folder(folder_path, real_width_mm, thresh=128,
                   largest_only=False, save_mask_preview=True):
    folder = Path(folder_path)
    if not folder.exists():
        raise FileNotFoundError(f"폴더가 존재하지 않습니다: {folder}")

    out_dir = folder / "area_measure_result"
    mask_dir = out_dir / "masks"
    out_dir.mkdir(parents=True, exist_ok=True)
    if save_mask_preview:
        mask_dir.mkdir(parents=True, exist_ok=True)

    rows = []
    for p in sorted(folder.iterdir()):
        if p.suffix.lower() not in EXTS:
            continue
        img = imread_unicode(p, cv2.IMREAD_UNCHANGED)
        if img is None:
            print(f"[WARN] 이미지 읽기 실패: {p.name}")
            continue

        # 회색조/이진화
        binary = binarize_to_black_foreground(img, thresh=thresh)

        # 가장 큰 성분만 사용할지
        if largest_only:
            binary = largest_component_mask(binary)
            note = "largest_only"
        else:
            note = "total_black"

        # 이미지 가로 픽셀 수
        h, w = binary.shape[:2]
        if w == 0:
            print(f"[WARN] 잘못된 폭(0) : {p.name}")
            continue

        # 픽셀 물리 크기(mm/px)와 픽셀 면적(mm^2/px)
        px_size_mm = real_width_mm / float(w)
        px_area_mm2 = px_size_mm * px_size_mm

        # 흰색(255) 픽셀 수 카운트
        black_count = int(np.sum(binary == 255))
        area_mm2 = black_count * px_area_mm2

        rows.append({
            "filename": p.name,
            "width_px": w,
            "height_px": h,
            "real_width_mm": real_width_mm,
            "px_size_mm": px_size_mm,
            "black_pixels": black_count,
            "area_mm2": area_mm2,
            "note": note
        })

        # 미리보기 저장 (원본이 컬러면 좌상 원본/우상 회색/좌하 마스크/우하 오버레이 같은 것도 가능)
        if save_mask_preview:
            # 마스크를 컬러로 만들어 빨간 오버레이
            vis = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
            overlay = imread_unicode(p, cv2.IMREAD_COLOR)
            if overlay is None:
                overlay = cv2.cvtColor((255 - binary), cv2.COLOR_GRAY2BGR)
            else:
                if overlay.shape[:2] != binary.shape[:2]:
                    overlay = cv2.resize(overlay, (w, h), interpolation=cv2.INTER_NEAREST)
                mask_fg = (binary == 0).astype(np.uint8)
                # 빨간색 오버레이 (알파 0.35)
                red = np.zeros_like(overlay)
                red[:, :, 2] = 255
                overlay = (overlay * 1.0).astype(np.float32)
                red = (red * 1.0).astype(np.float32)
                alpha = 0.35
                overlay[mask_fg == 1] = (1 - alpha) * overlay[mask_fg == 1] + alpha * red[mask_fg == 1]
                overlay = np.clip(overlay, 0, 255).astype(np.uint8)

            # 하단에 면적 텍스트 표시
            text = f"Area = {area_mm2:.3f} mm^2 ({black_count} px)"
            cv2.putText(overlay, text, (10, h - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (0, 0, 0), 3, cv2.LINE_AA)
            cv2.putText(overlay, text, (10, h - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (255, 255, 255), 1, cv2.LINE_AA)

            save_path = mask_dir / f"{p.stem}_preview.png"
            imwrite_unicode(save_path, overlay)

    # CSV 저장
    csv_path = out_dir / "areas_mm2.csv"
    with open(csv_path, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=[
            "filename", "width_px", "height_px", "real_width_mm",
            "px_size_mm", "black_pixels", "area_mm2", "note"
        ])
        writer.writeheader()
        for r in rows:
            writer.writerow(r)

    print(f"[OK] 처리 완료: {len(rows)}개 이미지")
    print(f"[INFO] 결과 CSV: {csv_path}")
    if SAVE_MASK_PREVIEW:
        print(f"[INFO] 미리보기 폴더: {mask_dir}")

if __name__ == "__main__":
    process_folder(
        FOLDER_PATH,
        REAL_WIDTH_MM,
        thresh=THRESH,
        largest_only=LARGEST_ONLY,
        save_mask_preview=SAVE_MASK_PREVIEW
    )


[OK] 처리 완료: 192개 이미지
[INFO] 결과 CSV: C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\250901_영상\피규어용\Aloe\aloe_ver2_300-700\Binary\area_measure_result\areas_mm2.csv
[INFO] 미리보기 폴더: C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\250901_영상\피규어용\Aloe\aloe_ver2_300-700\Binary\area_measure_result\masks
